# Import lib


In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt

# Read data

In [124]:
# df = pd.read_csv('../data.csv')
df = pd.read_csv('../../data.csv')

/var/folders/v3/t0psjdhs29g8j3cghfdfjd9w0000gn/T/ipykernel_43536/4239913280.py:2: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data.csv')


In [125]:
#check null
null_counts = df.isnull().sum()
print(null_counts)

id                       0
member_id                0
loan_amnt                0
funded_amnt              0
funded_amnt_inv          0
                     ...  
total_rev_hi_lim     67313
inq_fi              842681
total_cu_tl         842681
inq_last_12m        842681
default_ind              0
Length: 73, dtype: int64


# Discover and visualise the data

#### remove irrelevant features

In [126]:
remove_col = [
    'id',
    'member_id',
    'emp_title',
    'issue_d',
    'desc',
    'title',
    'zip_code',
    'addr_state',
    'earliest_cr_line',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'next_pymnt_d',
    'last_credit_pull_d',
    'collections_12_mths_ex_med',
    'mths_since_last_major_derog',
    'policy_code',

    # Vu Anh
    'purpose', 
    'sub_grade',
    'tot_coll_amt', 
    'tot_cur_bal', 
    'open_acc_6m', 
    'open_il_6m', 

    # Darrel
    # 'open_il_12m', 
    # 'open_il_24m', 
    # 'mths_since_rcnt_il', 
    # 'total_bal_il', 
    # 'il_util', 
    # 'open_rv_12m' ,
    # 'open_rv_24m', 

    # Vanness
    'max_bal_bc', 
    'all_util', 
    'total_rev_hi_lim', 
    'inq_fi', 
    'total_cu_tl', 
    'inq_last_12m'
]

In [127]:
df = df.drop(remove_col, axis=1)

In [128]:
np.shape(df)

(855969, 40)

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855969 entries, 0 to 855968
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   loan_amnt                  855969 non-null  int64  
 1   funded_amnt                855969 non-null  int64  
 2   funded_amnt_inv            855969 non-null  float64
 3   term                       855969 non-null  object 
 4   int_rate                   855969 non-null  float64
 5   installment                855969 non-null  float64
 6   grade                      855969 non-null  object 
 7   emp_length                 812908 non-null  object 
 8   home_ownership             855969 non-null  object 
 9   annual_inc                 855969 non-null  float64
 10  verification_status        855969 non-null  object 
 11  pymnt_plan                 855969 non-null  object 
 12  dti                        855969 non-null  float64
 13  delinq_2yrs                85

### Process missing value

In [130]:
# Display all attributes with missing values

null_columns = df.columns[df.isnull().any()] 
null_columns_result = df.isnull().any()[null_columns] 
null_columns_result

emp_length                   True
mths_since_last_delinq       True
mths_since_last_record       True
revol_util                   True
annual_inc_joint             True
dti_joint                    True
verification_status_joint    True
open_il_12m                  True
open_il_24m                  True
dtype: bool

#### Emp_length

In [131]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [132]:
label_encoder = LabelEncoder()

# Fit the LabelEncoder with unique values
label_encoder.fit(df['emp_length'].unique())

# Encode the attribute values
df['emp_length'] = label_encoder.transform(df['emp_length'])

In [133]:
df['emp_length'].unique()

array([ 1, 10,  0,  3,  8,  9,  4,  5,  6,  2,  7, 11])

#### mths_since_last_delinq 

In [134]:
df['mths_since_last_delinq'] = df['mths_since_last_delinq'].fillna(-1)

#### mths_since_last_record

In [135]:
df['mths_since_last_record'] = df['mths_since_last_record'].fillna(-1)

#### revol_util

In [136]:
imputer = SimpleImputer(strategy='mean')

revol_util = df['revol_util'].values.reshape(-1,1)

revol_util_imputed = imputer.fit_transform(revol_util)

df['revol_util'] = revol_util_imputed

#### annual_inc_joint

In [137]:
# aggregate annual_inc and annual_inc_joint
df.loc[df['application_type'] == 'JOINT', 'annual_inc'] = df.loc[df['application_type'] == 'JOINT', 'annual_inc_joint']
df = df.drop('annual_inc_joint', axis=1)

#### dti_joint 

In [138]:
df.loc[df['application_type'] == 'JOINT', 'dti'] = df.loc[df['application_type'] == 'JOINT', 'dti_joint']
df = df.drop('dti_joint', axis=1)

#### verification_status_joint 

In [139]:
df.loc[df['application_type'] == 'JOINT', 'verification_status'] = df.loc[df['application_type'] == 'JOINT', 'verification_status_joint']
df = df.drop('verification_status_joint', axis=1)

### Data binning

#### term

In [140]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [141]:
term = [' 36 months', ' 60 months']  # Unique values for encoding

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the grades
encoder.fit(term)

# Encode the 'grade' column in the DataFrame
df['term'] = encoder.transform(df['term'])

### grade

In [142]:
grades = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(grades)

df['grade'] = encoder.transform(df['grade'])

### verification_status

In [143]:
veri = ['Not Verified', 'Source Verified', 'Verified']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(veri)


df['verification_status'] = encoder.transform(df['verification_status'])

### pymnt_plan

In [144]:
plan = ['n', 'y']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(plan)

df['pymnt_plan'] = encoder.transform(df['pymnt_plan'])

### application_type

In [145]:
type = ['INDIVIDUAL', 'JOINT'] # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(type)

df['application_type'] = encoder.transform(df['application_type'])

#### initial_list_status

In [146]:

status = ['f', 'w'] # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(status)

df['initial_list_status'] = encoder.transform(df['initial_list_status'])

### Cleaning of the following

    'open_il_12m', 842681 missing values, can consider scrapping as there's no value
    'open_il_24m', 842681 missing values, can consider scrapping as there's no value
    'mths_since_rcnt_il', 
    'total_bal_il', 
    'il_util', 
    'open_rv_12m' ,
    'open_rv_24m', 

In [147]:
#check null
null_counts = df.isnull().sum()
print(null_counts)

loan_amnt                       0
funded_amnt                     0
funded_amnt_inv                 0
term                            0
int_rate                        0
installment                     0
grade                           0
emp_length                      0
home_ownership                  0
annual_inc                      0
verification_status             0
pymnt_plan                      0
dti                             0
delinq_2yrs                     0
inq_last_6mths                  0
mths_since_last_delinq          0
mths_since_last_record          0
open_acc                        0
pub_rec                         0
revol_bal                       0
revol_util                      0
total_acc                       0
initial_list_status             0
out_prncp                       0
out_prncp_inv                   0
total_pymnt                     0
total_pymnt_inv                 0
total_rec_prncp                 0
total_rec_int                   0
total_rec_late

In [148]:
df['open_il_12m'].unique()

array([nan,  0.,  1.,  4.,  2.,  3.,  5.,  6.,  8.,  7.,  9., 10., 12.])

### Generate new feature

In [149]:
# Generate the new feature
df['open_acc_rate'] = df['open_acc'] / df['total_acc']
# Swap the values and column names
df['open_acc_rate'], df['default_ind'] = df['default_ind'].copy(), df['open_acc_rate'].copy()
df.rename(columns={'open_acc_rate': 'default_ind', 'default_ind': 'open_acc_rate'}, inplace=True)

In [150]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,application_type,acc_now_delinq,open_il_12m,open_il_24m,open_acc_rate,default_ind
0,5000,5000,4975.0,0,10.65,162.87,1,1,RENT,24000.0,...,861.07,0.00,0.00,0.00,0,0,NaN,NaN,0.333333,0
1,2500,2500,2500.0,1,15.27,59.83,2,10,RENT,30000.0,...,435.17,0.00,117.08,1.11,0,0,NaN,NaN,0.750000,1
2,2400,2400,2400.0,0,15.96,84.33,2,1,RENT,12252.0,...,603.65,0.00,0.00,0.00,0,0,NaN,NaN,0.200000,0
3,10000,10000,10000.0,0,13.49,339.31,2,1,RENT,49200.0,...,2209.33,16.97,0.00,0.00,0,0,NaN,NaN,0.270270,0
4,3000,3000,3000.0,1,12.69,67.79,1,0,RENT,80000.0,...,1009.07,0.00,0.00,0.00,0,0,NaN,NaN,0.394737,0


In [151]:
np.shape(df)

(855969, 38)

### Visualize

In [152]:
# Create a correlation matrix
corr_matrix = df.corr()

# Select the correlation values with 'default_ind'
target_corr = corr_matrix['default_ind']

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(12, 12))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

ValueError: could not convert string to float: 'RENT'

### Attribute ranking

In [ ]:
# Take the absolute values of the correlation
abs_corr = target_corr.abs()

# Sort the absolute correlation values
sorted_corr = abs_corr.sort_values(ascending=False)

# Display the attribute ranking
attribute_ranking = sorted_corr.reset_index()
attribute_ranking.columns = ['Attribute', 'Absolute Correlation']
attribute_ranking